# **Deep Research with Agent Framework**

This notebook demonstrates an agentic research workflow using **Microsoft Agent Framework** to orchestrate a multi-agent system for comprehensive web-based research. Unlike notebooks 01-03, this implementation uses Agent Framework's workflow builder for more sophisticated agent coordination and routing logic.

## Key Features

- **Workflow Orchestration** - Uses Agent Framework's WorkflowBuilder for declarative agent coordination
- **Conditional Routing** - Implements switch-case logic for dynamic workflow paths based on peer review feedback
- **Pure Azure AI Agents** - All agents pre-created in Azure AI Foundry (same as notebook 03)
- **Automated Quality Control** - Peer review agent routes workflow to completion, revision, or data gathering
- **Workflow Visualization** - Generates SVG diagrams of the agent workflow

Built with Microsoft Agent Framework, Azure AI Projects, and Azure AI Agents

## Required Environment Variables

This notebook requires the following environment variables in your `.env` file:

```bash
# Azure AI Projects Configuration
PROJECT_ENDPOINT=your_azure_ai_project_endpoint

# Pre-created Agent IDs (must be created via 00_create_agents.py)
PlannerAgentID=your_planner_agent_id
BingSearchAgentID=your_bing_search_agent_id
SummaryAgentID=your_summary_agent_id
ResearchAgentID=your_research_agent_id
PeerReviewAgentMultiChoiceID=your_peer_review_multi_choice_agent_id
```

## Environment and Dependency Setup

Import necessary libraries and configure the environment for Agent Framework execution.

In [1]:
# Environment and dependency setup
import os
import dotenv
dotenv.load_dotenv(".env", override=True)

# Enable nested async event loops (required for Jupyter notebooks)
import nest_asyncio
nest_asyncio.apply()

# Agent Framework imports for workflow orchestration
from agent_framework import (
    WorkflowBuilder,
    Case,
    Default,
    WorkflowViz
)

# Project-specific imports
from common.data_models import NextAction
from maf.helper import save_report
from maf.update_agent_instructions import update_agent_instructions
from maf.agents import planner_agent, peer_review_agent_multi_choice, cleanup_all_agents
from maf.nodes import (
    search_executor,
    summary_executor,
    research_report_executor,
    to_routing_decision,
    get_next_action,
    handle_complete,
    handle_routing_error,
)

### Creating Azure AI Agents (One-time Setup)

The following cell will **create all Azure AI Agents** required for this workflow. You only need to run this cell **once** to create the agents, then save their IDs to your `.env` file.

After creating the agents, uncomment the fetch agents cell below and comment out this creation cell for subsequent runs.

In [2]:
# from common.create_azure_ai_agents import get_project_client, create_agents
# from maf.create_peer_review_agent_multi_choice import create_peer_review_agent_multi_choice

# project_client = get_project_client(os.getenv("PROJECT_ENDPOINT"))
# create_agents(project_client) # creates all agents except the peer review agent
# create_peer_review_agent_multi_choice(project_client) # creates the multi condition peer review agent

## Workflow Setup

The workflow orchestrates multiple agents in a research pipeline:

1. **PlannerAgent** → Creates research plan with subtopics and queries
2. **SearchExecutor** → Executes Bing searches for each query
3. **SummaryExecutor** → Summarizes search results
4. **ResearchReportExecutor** → Generates comprehensive report
5. **PeerReviewAgent** → Evaluates report quality and decides next action
6. **RoutingDecision** → Routes to completion, revision, or more data gathering

The peer review agent uses multi-choice output to determine the workflow path:
- `COMPLETE` → Workflow finishes, report is saved
- `REVISE_REPORT` → Loops back to research report executor
- `GATHER_MORE_DATA` → Loops back to search executor for additional information

In [3]:
# Update all agent instructions with current date and refined prompts
update_agent_instructions()

# Build the workflow with declarative edge definitions
workflow = (
    WorkflowBuilder()
    .set_start_executor(planner_agent)
    .add_edge(planner_agent, search_executor)
    .add_edge(search_executor, summary_executor)
    .add_edge(summary_executor, research_report_executor)
    .add_edge(research_report_executor, peer_review_agent_multi_choice)
    .add_edge(peer_review_agent_multi_choice, to_routing_decision)
    # Conditional routing based on peer review feedback
    .add_switch_case_edge_group(
        to_routing_decision,
        [
            Case(condition=get_next_action(NextAction.COMPLETE), target=handle_complete),
            Case(condition=get_next_action(NextAction.REVISE_REPORT), target=research_report_executor),
            Case(condition=get_next_action(NextAction.GATHER_MORE_DATA), target=search_executor),
            Default(target=handle_routing_error),
        ]
    )
    .build()
)

[2025-11-03 16:46:27 - c:\Users\povelf\AppData\Local\miniconda3\envs\deep-research\Lib\site-packages\agent_framework\_workflows\_validation.py:520 - WARNING] Cycle detected in the workflow graph involving: to_routing_decision -> PeerReviewAgentMultiChoice -> research_report_executor -> summary_executor -> search_executor -> to_routing_decision. Ensure termination or iteration limits exist.


## Workflow Visualization

Generate a visual representation of the agent workflow showing all nodes and routing paths.

In [4]:
print("Generating workflow visualization...")
viz = WorkflowViz(workflow)

try:
    svg_file = viz.export(format="svg", filename="workflow_graph_agent_framework.svg")
    print(f"SVG file saved to: {svg_file}")
except ImportError:
    print("Tip: Install 'viz' extra to export workflow visualization: pip install agent-framework[viz] --pre")

Generating workflow visualization...
SVG file saved to: workflow_graph_agent_framework.svg


## Research Query

Define your research topic or question below.

In [5]:
user_query="What are the differences between classical machine learning, deep learning and generative AI?"

## Execute Workflow

Run the complete research workflow. The workflow will:
1. Plan the research
2. Search for information
3. Summarize findings
4. Generate a report
5. Peer review and iterate until quality standards are met
6. Save the final report

The workflow automatically handles agent cleanup on completion or error.

In [6]:
try:
    events = await workflow.run(user_query)
    outputs = events.get_outputs()
    final_report = outputs[0]   
    save_report(final_report)
    
except Exception as e:
    print(f"Error during workflow execution: {e}")
    raise
finally:
    print("\n[Main] Cleaning up agent clients...")
    await cleanup_all_agents()

[ResearchPlanMiddleware] 🚀 Agent PlannerAgent starting...
[SearchExecutor] Initial search execution from planner
[SearchExecutor] Stored research plan in shared state (3 tasks, 6 queries)
[SearchExecutor] 🚀 Agent BingSearchAgent starting...
[SearchExecutor] 🚀 Agent BingSearchAgent starting...
[SearchExecutor] 🚀 Agent BingSearchAgent starting...
[SearchExecutor] 🚀 Agent BingSearchAgent starting...
[SearchExecutor] 🚀 Agent BingSearchAgent starting...
[SearchExecutor] 🚀 Agent BingSearchAgent starting...
[SearchExecutor] ✅ Agent BingSearchAgent completed in 22.26s
[SearchExecutor] ✅ Agent BingSearchAgent completed in 25.31s
[SearchExecutor] ✅ Agent BingSearchAgent completed in 33.59s
[SearchExecutor] ✅ Agent BingSearchAgent completed in 30.08s
[SearchExecutor] ✅ Agent BingSearchAgent completed in 24.57s
[SearchExecutor] ✅ Agent BingSearchAgent completed in 35.64s
[SearchExecutor] Completed 6 searches
[SummaryExecutor] Summarizing 3 subtopics...
[AgentMiddleware] 🚀 Agent SummaryAgent starti